In [ ]:
import geokit as gk
import numpy as np
import pandas as pd
from os.path import join
import matplotlib.pyplot as plt

# RegionMask
The RegionMask object is the geospatial representation of the given region, allows to combine raster and vector operations and many functions. 
The following shows only a few examples of these. 



Example for the Aachen-Region

In [ ]:
# define the srs
aachen_centered_srs = gk.srs.centeredLAEA(6.083, 50.775)

In [ ]:
# load the region mask from the passed shape file
aachen_rm = gk.RegionMask.load(
    gk._test_data_["aachenShapefile.shp"], pixelRes=20, srs=aachen_centered_srs
)
aachen_rm

In [ ]:
# Print information about the region mask
print("aachen_rm.extent", aachen_rm.extent)
# print("aachen_rm.srs", aachen_rm.srs)
print("aachen_rm.pixelWidth", aachen_rm.pixelWidth)
print("aachen_rm.pixelHeight", aachen_rm.pixelHeight)

## Region Mask and Raster-File

A RegionMask object allows raster data to be warped onto the RegionMask's characteristics (as in the RegionMask's resolution, extent, and spatial reference system). This create a numpy matrix, from which only the pixels which lie in the region can be easily extracted and operated on.


Example: Extract the GHI-data for Aachen from a raster-file for the GHI in Germany.


In [ ]:
ras = aachen_rm.warp(gk._test_data_["gsa-ghi-like.tif"], returnMatrix=False)

Draw Raster file in RegionMask.

In [ ]:
gk.drawRaster(ras, figsize=(6, 6))

Raster values: Average values and histogram

In [ ]:
# Get the average of raster values which are within the region mask
aachen_rm.warp(gk._test_data_["gsa-ghi-like.tif"])[aachen_rm.mask].mean()

In [ ]:
# plot histogramm of the values
plt.hist(aachen_rm.warp(gk._test_data_["gsa-ghi-like.tif"])[aachen_rm.mask])

## Region Mask and shp-file

Extract feature in a df.

In [ ]:
aachen_points = aachen_rm.extractFeatures(gk._test_data_["turbinePlacements.shp"])
aachen_points.head()

1. Example: Show features in the region mask.

In [ ]:
axh = aachen_rm.drawGeoms(aachen_points, figsize=(5, 6), hideAxis=True, color="b")
plt.show()

2. Example: Indicate features directly in region mask. Include a buffer around the features.

In [ ]:
# Indicate RegionMask pixels within 500 meters of these points
isNearToTurbine = aachen_rm.indicateFeatures(
    gk._test_data_["turbinePlacements.shp"], buffer=500
)
# Visualize it!
axh = aachen_rm.drawImage(
    isNearToTurbine,
    figsize=(5, 6),
    hideAxis=True,
    cmap="Reds",
)
plt.show()

## Export a raster-tif-file for the region 

In [ ]:
# Create a raster
example_data = np.ones_like(aachen_rm.mask)
aachen_rm.createRaster(data=example_data, output="./rm_raster.tif")

## Indicate areas with values

In [ ]:
# Show areas with GHI values > 2.9
tmp = aachen_rm.indicateValues(
    source=gk._test_data_["gsa-ghi-like.tif"],
    value=(2.9, None),
)

aachen_rm.drawImage(tmp, figsize=(6, 6))

In [ ]:
# Show areas with GHI values > 2.9 with buffer
tmp = aachen_rm.indicateValues(
    source=gk._test_data_["gsa-ghi-like.tif"], value=(2.9, None), buffer=100
)

aachen_rm.drawImage(tmp, figsize=(6, 6))